In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
from glob import glob
import matplotlib
import matplotlib.ticker as ticker
from matplotlib.patches import Polygon
import matplotlib.patches as mpatches
import os
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cv2
matplotlib.rcParams['figure.figsize'] = (13.33,5.5)
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
sns.set_palette('colorblind')

In [ ]:
data = pd.read_csv('data/processed/subset.csv')
fgs = pd.read_csv('data/processed/subset_fgs.csv')
fphis = pd.read_csv('data/processed/subset_fphis.csv')
Gs = np.load('data/processed/G_field.npy')
xx = np.load('data/processed/x_coords.npy')

In [ ]:
from scipy.stats import halfnorm

In [ ]:
def differences(fits,m_type,flip=False):
    prob1 = []
    prob2 = []
    diffs = []
    diffs2 = []
    for i in fits['incubation'].unique():
        a = fits[(fits['incubation']==i) & (fits['condition']=='CAF')][f'alpha_{m_type}_s']
        b = fits[(fits['incubation']==i) & (fits['condition']=='control')][f'alpha_{m_type}_s']
        c = (a.values-b.values)
        if flip:
            prob1.append((c<0).sum()/c.shape[0])
        else:
            prob1.append((c>0).sum()/c.shape[0])
        #print((c>0).sum()/c.shape[0])
        diffs.append(c)
        a = fits[(fits['incubation']==i) & (fits['condition']=='CAF')][f'alpha_{m_type}_sigma']
        b = fits[(fits['incubation']==i) & (fits['condition']=='control')][f'alpha_{m_type}_sigma']
        c = (a.values-b.values)
        #print((c>0).sum()/c.shape[0])
        if flip:
            prob2.append((c<0).sum()/c.shape[0])
        else:
            prob2.append((c>0).sum()/c.shape[0])
        diffs2.append(c)
    diffs = pd.DataFrame(diffs).T
    diffs.columns = np.unique(fits['incubation'])
    diffs['level'] = 'fov'
    diffs2 = pd.DataFrame(diffs2).T
    diffs2.columns = np.unique(fits['incubation'])
    diffs2['level'] = 'between'
    diff = pd.concat([diffs,diffs2])

    merged = diff.melt(id_vars=['level']).merge(pd.concat([
        pd.DataFrame({'level':'fov','variable':np.unique(fits['incubation']),'prob':prob1}),
        pd.DataFrame({'level':'between','variable':np.unique(fits['incubation']),'prob':prob2})]),on=['level','variable'])
    merged['type']=m_type
    return merged

probs_g = []
probs_phi = []
diffs = []


m1 = differences(data,'g')
m2 = differences(data,'phi')

probs_g = m1.groupby(['level','variable']).agg({'value':np.mean,'prob':np.mean,'type':'first'}).reset_index()
probs_phi = m2.groupby(['level','variable']).agg({'value':np.mean,'prob':np.mean,'type':'first'}).reset_index()

for i in [1,2,3]:
    su = data[data['repetition']==i]
    incubation = su[su['condition']=='CAF']['incubation'].values
    diff = su[su['condition']=='CAF'][['alpha_g','alpha_phi','alpha_g_sigma','alpha_phi_sigma']].reset_index()-su[su['condition']=='control'][['alpha_g','alpha_phi','alpha_g_sigma','alpha_phi_sigma']].reset_index()
    diff['incubation'] = incubation.astype(str)
    diff['repetition'] = i
    diffs.append(diff)

diffs = pd.concat(diffs)

In [ ]:
for i,j2 in diffs.groupby(['incubation']):
    print(i[0])
    for i2,j in j2.groupby(['repetition']):
        print('-- ',i2[0])
        for k in ['alpha_g', 'alpha_phi', 'alpha_g_sigma', 'alpha_phi_sigma']:
            prob = (j[k]>0).sum()/j.shape[0]
            mark = ''
            if 'phi' in k:
                mark = '*' if prob<0.05 else ''
            else:
                mark = '*' if prob>0.95 else ''
            print('---- {} {:.2f} {}'.format(k,prob,mark))

In [ ]:
from matplotlib.colors import to_rgba

In [ ]:
data['day_rep'] = data[['incubation','repetition']].astype(str).agg('-'.join,axis=1)
fgs['day_rep'] = fgs[['incubation','repetition']].astype(str).agg('-'.join,axis=1)
fphis['day_rep'] = fphis[['incubation','repetition']].astype(str).agg('-'.join,axis=1)

In [ ]:
day_repeat_order = data['day_rep'].unique()
day_labels = data['incubation'].unique()

# Calculate positions with gaps between days
positions = {}
spacing = 1  # Number of ticks to skip for each day
current_position = 0

# Create positions for each day-repeat combination with gaps between days
for day in day_labels:
    day_repeats = [dr for dr in day_repeat_order if dr.startswith(str(day))]
    for dr in day_repeats:
        positions[dr] = current_position
        current_position += 1
    current_position += spacing  # Add extra space after each day

data['x_position'] = data['day_rep'].map(positions)
fgs['x_position'] = fgs['day_rep'].map(positions)
fphis['x_position'] = fphis['day_rep'].map(positions)

fig,ax = plt.subplots()
ax = sns.violinplot(data=data,x='x_position',y='offset_g',hue='condition',
                   split=True,gap=0.3,inner=None,ax=ax,order=np.arange(12))
ax.set_xticks(list(positions.values()))
ax.set_xticklabels(['',1,'','',2,'','',3,'',],rotation=0)
# sns.violinplot(data=data[data['condition']=='model'],x='incubation',y='offset_g',hue='repetition',
#                    split=True,gap=1.,inner=None)

In [ ]:
for i,j in data.groupby(['incubation']):
    ca = j[j['condition']=='CAF']['offset_g'].values
    co = j[j['condition']=='control']['offset_g'].values
    diff = ca-co
    ca = j[j['condition']=='CAF']['offset_phi'].values
    co = j[j['condition']=='control']['offset_phi'].values
    diff2 = ca-co
    print(i,(diff>0).sum()/diff.shape[0],(diff2>0).sum()/diff2.shape[0])

In [ ]:
for i,j in data.groupby(['incubation','repetition']):
    ca = j[j['condition']=='CAF']['offset_g'].values
    co = j[j['condition']=='control']['offset_g'].values
    diff = ca-co
    ca = j[j['condition']=='CAF']['offset_phi'].values
    co = j[j['condition']=='control']['offset_phi'].values
    diff2 = ca-co
    print(i,(diff>0).sum()/diff.shape[0],(diff2>0).sum()/diff2.shape[0])

In [ ]:
for i,j in data.groupby(['incubation']):
    ca = j[j['condition']=='CAF']['offset_g'].values
    co = j[j['condition']=='control']['offset_g'].values
    diff = ca-co
    ca = j[j['condition']=='CAF']['offset_phi'].values
    co = j[j['condition']=='control']['offset_phi'].values
    diff2 = ca-co
    print(i,(diff>0).sum()/diff.shape[0],(diff2<0).sum()/diff2.shape[0])

In [ ]:
pairs = [[1,2],[1,3],[2,3]]
for p in pairs:
    ca = data[np.logical_and(data['condition']=='CAF',data['incubation']==p[0])]['offset_g'].values
    co = data[np.logical_and(data['condition']=='CAF',data['incubation']==p[1])]['offset_g'].values
    diff = ca-co
    ca = data[np.logical_and(data['condition']=='CAF',data['incubation']==p[0])]['offset_phi'].values
    co = data[np.logical_and(data['condition']=='CAF',data['incubation']==p[1])]['offset_phi'].values
    diff2 = ca-co
    print(i,(diff>0).sum()/diff.shape[0],(diff2>0).sum()/diff2.shape[0])

In [ ]:
pairs = [[1,2],[1,3],[2,3]]
for p in pairs:
    ca = data[np.logical_and(data['condition']=='control',data['incubation']==p[0])]['offset_g'].values
    co = data[np.logical_and(data['condition']=='control',data['incubation']==p[1])]['offset_g'].values
    diff = ca-co
    ca = data[np.logical_and(data['condition']=='control',data['incubation']==p[0])]['offset_phi'].values
    co = data[np.logical_and(data['condition']=='control',data['incubation']==p[1])]['offset_phi'].values
    diff2 = ca-co
    print(i,(diff>0).sum()/diff.shape[0],(diff2<0).sum()/diff2.shape[0])

In [ ]:
data['offset_phi_deg'] = np.rad2deg(data['offset_phi_s'])

In [ ]:
sigmoid = lambda x: 1/(1 + np.exp(-x))
trans = lambda x: np.arcsin(sigmoid(x))
# fphis['trans'] = np.rad2deg(trans(fphis['0']))
fphis['trans'] = np.rad2deg(fphis['0'])

In [ ]:

f_size = 15
f_size2 = 20
f_size3 = 30
fig,axes = plt.subplots(2,2,figsize=(13,5))

palette1 = sns.color_palette('blend:#1f2e47,#a1c9f4',3)
palette2 = sns.color_palette('blend:#6a3415,#ffb482',3)
alpha = 0.9


# for idx,i in enumerate([1,2,3]):
#     palette = list([palette1[idx],palette2[idx]])
#     sns.violinplot(data=fgs[fgs['repetition']==i],x='incubation',y='0',hue='condition',
#                    split=True,gap=0.1,inner=None,ax=ax[0],alpha=alpha,palette=palette,fill=True)
#     # for violin in ax[0].collections[::2]:
#     #     violin.set_alpha(0.3)
#     sns.violinplot(data=fphis[fphis['repetition']==i],x='incubation',y='0',hue='condition',
#                    split=True,gap=0.1,inner=None,ax=ax[1],alpha=alpha,palette=palette,fill=True)

ax = axes[0]

ax[0] = sns.violinplot(data=fgs,x='x_position',y='0',hue='condition',
                   split=True,gap=0.3,inner=None,ax=ax[0],order=np.arange(12))
ax[0].set_xticks(list(positions.values()))
ax[0].set_xticklabels(['',1,'','',2,'','',3,'',],rotation=0)

ax[0].set_ylim([0,130])

ax[1] = sns.violinplot(data=fphis,x='x_position',y='trans',hue='condition',
                   split=True,gap=0.3,inner=None,ax=ax[1],order=np.arange(12))
ax[1].set_xticks(list(positions.values()))
ax[1].set_xticklabels(['',1,'','',2,'','',3,'',],rotation=0)

ax[1].legend().remove()

ax3 = axes[1]

red_patch = mpatches.Patch(color=palette1[1], label='CAF')
blue_patch = mpatches.Patch(color=palette2[1], label='control')
ax[0].legend(handles=[red_patch, blue_patch],bbox_to_anchor=[0.75,1.0])

plt.legend(handles=[red_patch, blue_patch],bbox_to_anchor=[0.75,1.0])


# for idx,i in enumerate([1,2,3]):
#     palette = list([palette1[idx],palette2[idx]])
#     sns.violinplot(data=data[data['repetition']==i],x='incubation',y='offset_g',hue='condition',
#                    split=True,gap=0.1,inner=None,ax=ax3[0],alpha=alpha,palette=palette,fill=True)
#     sns.violinplot(data=data[data['repetition']==i],x='incubation',y='offset_phi',hue='condition',
#                    split=True,gap=0.1,inner=None,ax=ax3[1],alpha=alpha,palette=palette,fill=True)

ax3[0] = sns.violinplot(data=data,x='x_position',y='offset_g',hue='condition',
                   split=True,gap=0.3,inner=None,ax=ax3[0],order=np.arange(12))
ax3[0].set_xticks(list(positions.values()))
ax3[0].set_xticklabels(['',1,'','',2,'','',3,'',],rotation=0)

ax3[1] = sns.violinplot(data=data,x='x_position',y='offset_phi_deg',hue='condition',
                   split=True,gap=0.3,inner=None,ax=ax3[1],order=np.arange(12))
ax3[1].set_xticks(list(positions.values()))
ax3[1].set_xticklabels(['',1,'','',2,'','',3,'',],rotation=0)

ax3[0].legend().remove()
ax3[1].legend().remove()


ax3[0].set_ylabel(r'$|G^*|$ [Pa]',fontsize=f_size)
ax3[1].set_ylabel(r'$\phi$ [Degree]',fontsize=f_size)

ax3[0].yaxis.set_label_coords(-0.1,0.6)




ax[0].text(0.02,0.85,'A',transform=ax[0].transAxes,fontsize=f_size3)
ax3[0].text(0.02,0.85,'B',transform=ax3[0].transAxes,fontsize=f_size3)

ax[1].text(0.02,0.85,'C',transform=ax[1].transAxes,fontsize=f_size3)
ax3[1].text(0.02,0.85,'D',transform=ax3[1].transAxes,fontsize=f_size3)

ax3[0].set_xlabel('incubation [days]',fontsize=f_size)
ax3[1].set_xlabel('incubation [days]',fontsize=f_size)


ax[0].set_xlabel('',fontsize=f_size)
ax[1].set_xlabel('',fontsize=f_size)

ax[0].set_ylabel(r'$|G^*|$ [Pa]',fontsize=f_size)
ax[1].set_ylabel(r'$\phi$ [Degree]',fontsize=f_size)

#ax[1].legend().remove()


f_tick = 15
#ax[0].xaxis.set_tick_params(labelsize=f_tick)
ax[0].set_ylim(0,150)

fig.savefig(f'results/results_separate',bbox_inches='tight',dpi=300)

In [ ]:
fig = plt.figure(layout='constrained', figsize=(15, 4*2))
subfigs = fig.subfigures(1, 2, wspace=0.1,hspace=0.1)
subfig_left = subfigs[0].subfigures(2,1)
subfig_right = subfigs[1].subfigures(2,1,height_ratios=[1.5,1])

f_size = 15
f_size2 = 20
f_size3 = 30
ax = subfig_left[0].subplots(2,1)

palette1 = sns.color_palette('blend:#1f2e47,#a1c9f4',3)
palette2 = sns.color_palette('blend:#6a3415,#ffb482',3)
alpha = 0.9


for idx,i in enumerate([1,2,3]):
    palette = list([palette1[idx],palette2[idx]])
    sns.violinplot(data=fgs[fgs['repetition']==i],x='incubation',y='0',hue='condition',
                   split=True,gap=0.1,inner=None,ax=ax[0],alpha=alpha,palette=palette,fill=True)
    for violin in ax[0].collections[1::2]:
        violin.set_alpha(0.5)
    for violin in ax[0].collections[::2]:
        violin.set_alpha(0.5)
    sns.violinplot(data=fphis[fphis['repetition']==i],x='incubation',y='trans',hue='condition',
                   split=True,gap=0.1,inner=None,ax=ax[1],alpha=alpha,palette=palette,fill=True)
    for violin in ax[1].collections[1::2]:
        violin.set_alpha(0.5)
    for violin in ax[1].collections[::2]:
        violin.set_alpha(0.5)
    
ax[0].set_ylim([0,130])
ax[0].legend().remove()

ax3 = subfig_left[1].subplots(2,1)

red_patch = mpatches.Patch(color=palette1[1], label='CAF')
blue_patch = mpatches.Patch(color=palette2[1], label='control')
ax[0].legend(handles=[red_patch, blue_patch],bbox_to_anchor=[0.75,1])

plt.legend(handles=[red_patch, blue_patch],bbox_to_anchor=[0.75,1])


for idx,i in enumerate([1,2,3]):
    palette = list([palette1[idx],palette2[idx]])
    sns.violinplot(data=data[data['repetition']==i],x='incubation',y='offset_g',hue='condition',
                   split=True,gap=0.1,inner=None,ax=ax3[0],alpha=alpha,palette=palette,fill=True)
    for violin in ax3[1].collections[1::2]:
        violin.set_alpha(0.5)
    sns.violinplot(data=data[data['repetition']==i],x='incubation',y='offset_phi_deg',hue='condition',
                   split=True,gap=0.1,inner=None,ax=ax3[1],alpha=alpha,palette=palette,fill=True)
    for violin in ax3[1].collections[1::2]:
        violin.set_alpha(0.5)
ax3[0].legend().remove()
ax3[1].legend().remove()


ax3[0].set_ylabel(r'$|G^*|$ [Pa]',fontsize=f_size)
ax3[1].set_ylabel(r'$\phi$ [Degree]',fontsize=f_size,labelpad=20)

ax3[0].yaxis.set_label_coords(-0.1,0.6)

ax2 = subfig_right[0].subplots(2,2)


for idx2,j in enumerate(['g','phi']):
    for idx,i in enumerate([f'alpha_{j}',f'alpha_{j}_sigma']):
        p = sns.violinplot(data=diffs,x=i,y='incubation',split=True,ax=ax2[idx2,idx],color='black',fill=False,inner=None)

        for violin in p.collections:
            path = violin.get_paths()[0]
            vertices = path.vertices
            
            shaded_vertices = vertices[(vertices[:, 0] >= 0)]
            
            polygon = Polygon(shaded_vertices, facecolor="grey", alpha=0.7,zorder=-1)
            p.add_patch(polygon)

le = probs_g[probs_g['level']=='fov']['prob'].values.shape[0]
for idx,i in enumerate(probs_g[probs_g['level']=='fov']['prob'].values):
    ax2[0,0].text(0.8,1-idx/le-0.1,f'{int(np.round(i*100))}%',transform=ax2[0,0].transAxes,fontsize=f_size)

for idx,i in enumerate(probs_phi[probs_phi['level']=='fov']['prob'].values):
    ax2[1,0].text(0.8,1-idx/le-0.1,f'{int(np.round(i*100))}%',transform=ax2[1,0].transAxes,fontsize=f_size)

for idx,i in enumerate(probs_g[probs_g['level']=='between']['prob'].values):
    ax2[0,1].text(0.8,1-idx/le-0.1,f'{int(np.round(i*100))}%',transform=ax2[0,1].transAxes,fontsize=f_size)

for idx,i in enumerate(probs_phi[probs_phi['level']=='between']['prob'].values):
    ax2[1,1].text(0.8,1-idx/le-0.1,f'{int(np.round(i*100))}%',transform=ax2[1,1].transAxes,fontsize=f_size)


ax2[0,0].text(0.05,0.95,'B',transform=ax2[0,0].transAxes,fontsize=f_size3)
ax2[0,1].text(0.05,0.95,'C',transform=ax2[0,1].transAxes,fontsize=f_size3)

ax4 = subfig_right[1].subplots(1,2)
back_img = cv2.imread('data/processed/img_00.png')

for (idx,fun) in zip([0,1],[np.nanmean,np.nanstd]):
    ax4[idx].imshow(back_img,extent=(0,1,0,1),origin='lower')
    im = ax4[idx].imshow(fun(Gs,axis=0),extent=(0,1,0,1),cmap='jet',origin='lower',alpha=0.3)
    ax4[idx].scatter(*xx.T,alpha=0.6,color='black')
    ax4[idx].set_xlabel('',fontsize=f_size)
    ax4[idx].set_ylabel('',fontsize=f_size)

    ax4[idx].tick_params(axis='both', which='major',bottom=False,top=False,labelbottom=False,labeltop=False)
    ax4[idx].set_axis_off()
    divider = make_axes_locatable(ax4[idx])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = fig.colorbar(im, cax=cax, orientation='vertical')

cbar.set_label(r'$|G^*|$ [Pa]',fontsize=f_size)




ax4[0].set_title(r'$\mu$',fontsize=f_size)
ax4[1].set_title(r'$\sigma$',fontsize=f_size)
ax4[0].text(-0.1,1.01,'E',transform=ax4[0].transAxes,fontsize=f_size3)
ax[0].text(0.02,0.85,'A',transform=ax[0].transAxes,fontsize=f_size3)
ax3[0].text(0.02,0.85,'D',transform=ax3[0].transAxes,fontsize=f_size3)

ax3[1].set_xlabel('incubation [days]',fontsize=f_size)
ax3[0].set_xlabel('')


ax[0].set_xlabel('',fontsize=f_size)
ax[1].set_xlabel('incubation [days]',fontsize=f_size)

ax[0].set_ylabel(r'$|G^*|$ [Pa]',fontsize=f_size)
ax[1].set_ylabel(r'$\phi$ [Degree]',fontsize=f_size,labelpad=10)

ax[1].legend().remove()

ax2[0,0].set_xlabel(r'')
ax2[0,1].set_xlabel(r'')
ax2[0,0].set_ylabel('incubation [days]',fontsize=f_size)

ax2[1,0].set_xlabel(r'$\alpha_\mu$ / material heterogeneity',fontsize=f_size)
ax2[1,1].set_xlabel(r'$\alpha_\sigma$ / Variability between FOVs',fontsize=f_size)
ax2[1,0].set_ylabel('incubation [days]',fontsize=f_size)

ax2[0,1].set_ylabel('')
ax2[1,1].set_ylabel('')

ax2[0,0].text(-0.4,0.5,r'$|G^*|$',transform=ax2[0,0].transAxes,fontsize=f_size)
ax2[1,0].text(-0.35,0.5,r'$\phi$',transform=ax2[1,0].transAxes,fontsize=f_size)


f_tick = 15
#ax[0].xaxis.set_tick_params(labelsize=f_tick)
ax[0].set_ylim(0,150)

fig.savefig(f'results/results.png',bbox_inches='tight',dpi=300)